In [27]:
import pickle
def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer

def calculate_wer_from_files(reference_file, hypothesis_file):
    with open(reference_file, 'rb') as ref_file:
        references = pickle.load(ref_file)
    with open(hypothesis_file, 'rb') as hyp_file:
        hypothesiss = pickle.load(hyp_file)
    if len(hypothesiss) != 100 :
        hypothesiss = hypothesiss[0]
    
    error = [calculate_wer(reference, hypothesis) for reference, hypothesis in zip(references, hypothesiss)]
    return error

In [31]:
import glob 
import pickle
import pandas as pd
water_marked_files = sorted(glob.glob('Dataset/Watermarked/en/*/*'))
attacked_files = sorted(glob.glob('Dataset/Attacked/*/*/*'))

In [132]:
d= {}
for wfile in water_marked_files:
    for afile in attacked_files:
        key = afile.split('/')[2]+' '+ afile.split('/')[-2]  + ' '
        if 'rephrased'  in afile or 'dipper' in afile:
           key+= "".join(afile.split('/')[-1].split('_')[-1]) 
        key = key.lower().replace('.pkl' , '').replace('dipper' , 'rephrased').replace('sirrephrased' , 'rephrased').title()
        key = tuple(key.split(' '))
        if ('sir' in wfile.lower() and 'sir' in afile.lower()):
             d[key] = calculate_wer_from_files(wfile, afile)
        elif('semantics' not in wfile.lower() and 'semantics' not in afile.lower()):
               d[key] = calculate_wer_from_files(wfile, afile)
        else:
                d[key] = calculate_wer_from_files(wfile, afile)
      

In [133]:
df = pd.DataFrame(d).describe()
df = df.sort_index(axis = 1)

In [134]:
styled_df = df.style.set_table_styles([
    {'selector': 'th.level0',
     'props': [('text-align', 'center')]
    },
    {'selector': 'th',
     'props': [('background-color', '#f4f4f9'), ('color', 'black')]
    },
    {'selector': 'td',
     'props': [('background-color', '#f9f9f9')]
    },
    # Add horizontal lines after each row
    {'selector': 'tr',
     'props': [('border-bottom', '1px solid black')]
    },
     {'selector': 'td',
     'props': [('border-bottom', '1px solid black')]
    }
], overwrite=True)

In [135]:
styled_df